In [20]:
import imp
import random
import CustomDataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from torchvision import datasets, transforms

In [21]:
imp.reload(CustomDataset)
import CustomDataset

In [22]:
dataset_dic = CustomDataset.load_all_datasets()


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [23]:
dataset_dic.keys()

dict_keys(['MNIST_linear_unbalanced_1', 'CIFAR10_linear_unbalanced_1', 'MNIST_linear_unbalanced_2', 'CIFAR10_linear_unbalanced_2', 'MNIST_balanced', 'CIFAR10_balanced', 'MNIST_expo_unbalance', 'CIFAR10_expo_unbalance'])

In [24]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [25]:
n_epochs = 3
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 50

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [26]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), 'results/model.pth')
            torch.save(optimizer.state_dict(), 'results/optimizer.pth')

def test():
    network.eval()
    test_loss = 0
    correct = 0
    cm = np.zeros((10,10))
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
            cm = cm + confusion_matrix(np.array(target), np.array(pred), labels=[0,1,2,3,4,5,6,7,8,9])
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 
        100. * correct / len(test_loader.dataset)))
    print(cm.astype('int'))
    
    tp = np.max(np.eye(10)*cm,1)
    tp_and_fn = np.sum(cm, axis=1)
    tp_and_fp = np.sum(cm, axis=0)
    
    print("percision:")
    print(tp / (tp_and_fn+0.001))
    print("recall:")
    print(tp / (tp_and_fp+0.001))

In [27]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [28]:
train_loader = torch.utils.data.DataLoader((dataset_dic['MNIST_balanced']),batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=False, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [29]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [30]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

/afs/inf.ed.ac.uk/user/s16/s1636732/miniconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

[[   0  965    0    0   15    0    0    0    0    0]
 [   0 1135    0    0    0    0    0    0    0    0]
 [   0 1027    0    0    5    0    0    0    0    0]
 [   0 1010    0    0    0    0    0    0    0    0]
 [   0  980    0    0    2    0    0    0    0    0]
 [   0  885    0    0    7    0    0    0    0    0]
 [   0  922    0    0   36    0    0    0    0    0]
 [   0 1028    0    0    0    0    0    0    0    0]
 [   0  970    0    0    4    0    0    0    0    0]
 [   0 1002    0    0    7    0    0    0    0    0]]
percision:
[0.         0.99999912 0.         0.         0.00203666 0.
 0.         0.         0.         0.        ]
recall:
[0.         0.11436919 0.         0.         0.02631544 0.
 0.         0.         0.         0.        ]
Train Epoch: 1 [0/27500 (0%)]	Loss: 2.372260
Train Epoch: 1 [3200/27500 (12%)]	Loss: 2.263137
Train Epoch: 1 [6400/27500 (23%)]	Loss: 2.013866
Train Epoch: 1 [9600/27500 (35%)]	Loss:

In [31]:
train_loader = torch.utils.data.DataLoader((dataset_dic['MNIST_linear_unbalanced_1']),batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=False, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [32]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

/afs/inf.ed.ac.uk/user/s16/s1636732/miniconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. loss: 2.3067, Accuracy: 527/10000 (5%)

[[   0  149    0    0    0    0  527  304    0    0]
 [   0   16    0    0    0    0 1086   33    0    0]
 [   0   51    0    0    0    0  874  107    0    0]
 [   0   29    0    0    0    0  896   85    0    0]
 [   0  148    0    0    0    0  754   80    0    0]
 [   0  101    0    0    0    0  587  204    0    0]
 [   0  455    0    0    0    0  453   50    0    0]
 [   0   36    0    0    0    0  934   58    0    0]
 [   0  161    0    0    0    0  769   44    0    0]
 [   0  201    0    0    0    0  750   58    0    0]]
percision:
[0.         0.0140969  0.         0.         0.         0.
 0.47285963 0.05642018 0.         0.        ]
recall:
[0.         0.01187824 0.         0.         0.         0.
 0.0593709  0.05669594 0.         0.        ]
Train Epoch: 1 [0/27500 (0%)]	Loss: 2.315684
Train Epoch: 1 [3200/27500 (12%)]	Loss: 2.106379
Train Epoch: 1 [6400/27500 (23%)]	Loss: 1.793035
Train Epoch: 1 [9600/27500 (35%)]	Loss: 1

In [33]:
np.sum(np.array(dataset_dic['MNIST_linear_unbalanced_1'].targets)== 9)

4500

In [34]:
train_loader = torch.utils.data.DataLoader((dataset_dic['MNIST_expo_unbalance']),batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=False, download=False,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [35]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

/afs/inf.ed.ac.uk/user/s16/s1636732/miniconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. loss: 2.3109, Accuracy: 741/10000 (7%)

[[  0  90   0 364   0 524   0   0   2   0]
 [  0 256   0   9   0 870   0   0   0   0]
 [  0  85   0   6   0 940   0   0   1   0]
 [  0 242   0  38   0 717   0   0  13   0]
 [  0 559   0  23   3 387   0   0  10   0]
 [  0 192   0 248   0 441   0   0  11   0]
 [  0 193   0  78   0 679   0   0   8   0]
 [  0 607   0  11   0 404   0   0   6   0]
 [  0  66   0  12   0 893   0   0   3   0]
 [  0 448   0  23   3 515   0   0  20   0]]
percision:
[0.         0.22555046 0.         0.03762373 0.00305499 0.49439406
 0.         0.         0.00308008 0.        ]
recall:
[0.         0.09349887 0.         0.04679797 0.49991668 0.06923076
 0.         0.         0.04053999 0.        ]
Train Epoch: 1 [0/27500 (0%)]	Loss: 2.370296
Train Epoch: 1 [3200/27500 (12%)]	Loss: 2.194833
Train Epoch: 1 [6400/27500 (23%)]	Loss: 1.805672
Train Epoch: 1 [9600/27500 (35%)]	Loss: 1.272083
Train Epoch: 1 [12800/27500 (47%)]	Loss: 1.134211
Train Epoch: 1 [16000/2750

In [36]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv1_bn = nn.BatchNorm2d(32)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv4_bn = nn.BatchNorm2d(256)
        
        self.fc1 = nn.Linear(1024, 512)
        self.fc1_bn= nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 128)
        self.fc2_bn= nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1_bn(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_bn(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_bn(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_bn(self.conv4(x)), 2))
        x = x.view(-1, 1024)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2_bn(self.fc2(x)))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return F.log_softmax(x)

In [37]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

train_loader = torch.utils.data.DataLoader((dataset_dic['CIFAR10_balanced']),batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform),
  batch_size=batch_size_test, shuffle=True)

network = Net()
optimizer = optim.Adam(network.parameters())
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

Files already downloaded and verified


In [38]:
n_epochs = 100
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

/afs/inf.ed.ac.uk/user/s16/s1636732/miniconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. loss: 2.3038, Accuracy: 1001/10000 (10%)

[[   0    0    2    0    0    0    0    0  998    0]
 [   0    0    0    0    0    0    0    0 1000    0]
 [   0    0    1    0    0    0    0    0  999    0]
 [   0    0    0    0    0    0    0    0 1000    0]
 [   0    0    0    0    0    0    0    0 1000    0]
 [   0    0    0    0    0    0    0    0 1000    0]
 [   0    0    1    0    0    0    0    0  999    0]
 [   0    0    0    0    0    0    0    0 1000    0]
 [   0    0    0    0    0    0    0    0 1000    0]
 [   0    0    0    0    0    0    0    0 1000    0]]
percision:
[0.00000e+00 0.00000e+00 9.99999e-04 0.00000e+00 0.00000e+00 0.00000e+00
 0.00000e+00 0.00000e+00 9.99999e-01 0.00000e+00]
recall:
[0.         0.         0.24993752 0.         0.         0.
 0.         0.         0.10004001 0.        ]
Train Epoch: 1 [0/27500 (0%)]	Loss: 2.367719
Train Epoch: 1 [3200/27500 (12%)]	Loss: 1.823797
Train Epoch: 1 [6400/27500 (23%)]	Loss: 1.507494
Train Epoch: 1 [9600/

Train Epoch: 7 [22400/27500 (81%)]	Loss: 0.570581
Train Epoch: 7 [25600/27500 (93%)]	Loss: 0.694406

Test set: Avg. loss: 0.8140, Accuracy: 7321/10000 (73%)

[[816  13  17  19  15   1   0  12  83  24]
 [ 13 871   5   4   2   1   2   2  32  68]
 [111   8 632  49  87  17  16  47  29   4]
 [ 35   8  90 556  95  80  24  55  34  23]
 [ 29   3  49  44 776  12   4  61  21   1]
 [ 19   4  74 211  67 489   8  99  19  10]
 [ 20   6  96  93 106  10 626  11  22  10]
 [ 20   3  21  30  54  18   0 837   9   8]
 [ 41  17   5   6   7   1   1   3 904  15]
 [ 48  68   5   6   5   0   1  14  39 814]]
percision:
[0.81599918 0.87099913 0.63199937 0.55599944 0.77599922 0.48899951
 0.62599937 0.83699916 0.9039991  0.81399919]
recall:
[0.70833272 0.870129   0.63581425 0.54616842 0.6392087  0.77742325
 0.91788722 0.7335664  0.75838863 0.83316189]
Train Epoch: 8 [0/27500 (0%)]	Loss: 0.360334
Train Epoch: 8 [3200/27500 (12%)]	Loss: 0.290642
Train Epoch: 8 [6400/27500 (23%)]	Loss: 0.333529
Train Epoch: 8 [9600/27

KeyboardInterrupt: 

In [39]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

train_loader = torch.utils.data.DataLoader((dataset_dic['CIFAR10_linear_unbalanced_1']),batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform),
  batch_size=batch_size_test, shuffle=True)

network = Net()
optimizer = optim.Adam(network.parameters())
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

Files already downloaded and verified


In [40]:
# distribtion [ 500, 4000, 3500, 3000, 1500, 2500, 5000, 2000, 1000, 4500]
n_epochs = 100
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

/afs/inf.ed.ac.uk/user/s16/s1636732/miniconda3/envs/mlp/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Avg. loss: 2.3032, Accuracy: 1000/10000 (10%)

[[   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]
 [   0    0 1000    0    0    0    0    0    0    0]]
percision:
[0.       0.       0.999999 0.       0.       0.       0.       0.
 0.       0.      ]
recall:
[0.         0.         0.09999999 0.         0.         0.
 0.         0.         0.         0.        ]
Train Epoch: 1 [0/27500 (0%)]	Loss: 2.507617
Train Epoch: 1 [3200/27500 (12%)]	Loss: 1.515961
Train Epoch: 1 [6400/27500 (23%)]	Loss: 1.479474
Train Epoch: 1 [9600/27500 (35%)]	Loss: 1.235299
Train Ep

Train Epoch: 7 [22400/27500 (81%)]	Loss: 0.395281
Train Epoch: 7 [25600/27500 (93%)]	Loss: 0.645254

Test set: Avg. loss: 0.9127, Accuracy: 7088/10000 (70%)

[[374  33 311  37  40  15  17  29  54  90]
 [  1 899   5   5   1   6  10   5   6  62]
 [  4   4 754  35  31  67  69  20   4  12]
 [  1   8 102 510  33 210  83  25   2  26]
 [  1   2 138  74 535  59  99  76   7   9]
 [  1   4  51 109  13 722  48  42   1   9]
 [  0   4  42  34   5  19 888   3   2   3]
 [  2   6  34  28  29 106  15 765   3  12]
 [ 35  42  64  24   4   9  14   8 758  42]
 [  5  63   7   8   1   6   9   7  11 883]]
percision:
[0.37399963 0.8989991  0.75399925 0.50999949 0.53499947 0.72199928
 0.88799911 0.76499924 0.75799924 0.88299912]
recall:
[0.88207339 0.84413066 0.49999967 0.59027709 0.77312027 0.59228828
 0.70926461 0.78061145 0.89386687 0.76916309]
Train Epoch: 8 [0/27500 (0%)]	Loss: 0.336521
Train Epoch: 8 [3200/27500 (12%)]	Loss: 0.305963
Train Epoch: 8 [6400/27500 (23%)]	Loss: 0.354917
Train Epoch: 8 [9600/27

KeyboardInterrupt: 

In [ ]:
# percision:
# balance:   [0.7899 0.9029 0.5999 0.6079 0.6689 0.5459 0.8569 0.8789 0.7979 0.8489]
# unbalance:
# recall:
# balance:   [0.7692 0.8376 0.7317 0.5795 0.7295 0.7531 0.7748 0.6589 0.8711 0.8258]
# unbalance: 

In [ ]:
np.sum(np.array(dataset_dic['CIFAR10_linear_unbalanced_1'].targets)==0)

In [ ]:
np.sum(np.load("./data/CIFAR10_linear_unbalanced_1.npy") == 0)